## Transfer learning pretrained vgg16 network on cats and dogs
<img src="https://static.pexels.com/photos/126407/pexels-photo-126407.jpeg" width="200" height="300" />
<img src="https://d1wn0q81ehzw6k.cloudfront.net/additional/thul/media/7fbd0b08750fa69f?w=890&h=590&crop=1" width="200" height="300" />


Longhow


### Import neccesary stuff

In [1]:
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Flatten
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential


Using TensorFlow backend.


In [2]:
#base_model = InceptionV3(weights='imagenet', include_top=False)
base_model = VGG16(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- 2 classes dogs and cats
predictions = Dense(2, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',  metrics =['accuracy'])

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

#### Generate image data generators that can iterate through a directory of images

In [5]:
train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()
IM_H = 224
IM_W = 224

In [6]:
train_generator = train_datagen.flow_from_directory(
        'C:/Users/llam/OneDrive - RTL Nederland B.V/Personal/kerastest/train',  
        target_size=(IM_H, IM_W), 
        batch_size = 64,
        class_mode='categorical')

Found 2115 images belonging to 2 classes.


In [7]:
validation_generator = test_datagen.flow_from_directory(
        'C:/Users/llam/OneDrive - RTL Nederland B.V/Personal/kerastest/test',  
        target_size=(IM_H, IM_W), 
        batch_size = 64,
        class_mode='categorical')

Found 398 images belonging to 2 classes.


In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch = 32,
        epochs = 2,
        validation_data = validation_generator,
        validation_steps =6
)

Epoch 1/2
 5/32 [===>..........................] - ETA: 1596s - loss: 6.2538 - acc: 0.5625

### model save

In [11]:
from keras.models import load_model

model.save('my_finetuned_dogscats.h5')  

#### prediction using the model

In [17]:
from keras.applications.vgg16 import preprocess_input
import numpy as np
img_path = 'C:/Users/llam/OneDrive - RTL Nederland B.V/Personal/kerastest/cat.703.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

model.predict(x)

array([[  1.00000000e+00,   1.24319568e-16]], dtype=float32)

In [18]:
img_path = 'C:/Users/llam/OneDrive - RTL Nederland B.V/Personal/kerastest/dog.2908.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

model.predict(x)

array([[  1.00000000e+00,   3.00543679e-28]], dtype=float32)